A rough copy of https://blog.paperspace.com/writing-lenet5-from-scratch-in-python/

In [1]:
import numpy as np
from PIL import Image
import torch
from torch import nn
import torch.nn.functional as F
from datasets import fetch_mnist
from helpers import train, evaluate
torch.manual_seed(1337)

In [2]:
class LeNet5(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 6, 5),
            nn.BatchNorm2d(6),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, 5),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),
        )
        self.fc1 = nn.Linear(400, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def __call__(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.shape[0], -1)
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
        return out

In [3]:
def transform(x):
    x = [Image.fromarray(xx).resize((32, 32)) for xx in x]
    x = np.stack([np.asarray(xx) for xx in x], axis=0)
    x = x.reshape(-1, 1, 32, 32)
    return x

In [4]:
X_train, Y_train, X_test, Y_test = fetch_mnist()
X_train = X_train.reshape(-1, 28, 28) / 255.0
X_test = X_test.reshape(-1, 28, 28) / 255.0

model = LeNet5()
optimizer = torch.optim.Adam(model.parameters(), lr=0.002)
train(model, X_train, Y_train, optimizer, 1000, BS=128, transform=transform)
evaluate(model, X_test, Y_test, transform=transform)

100%|██████████| 79/79 [00:00<00:00, 95.71it/s]

test set accuracy is 0.9849
